In [4]:
import math
import random
import time
from jupyterthemes import jtplot
jtplot.style()

import gym
gym.logger.set_level(40)
import numpy as np

import nnabla as nn
import nnabla.logger as logger
import nnabla.functions as F
import nnabla.parametric_functions as PF
import nnabla.solver as S
from nnabla.contrib.context import extension_context
from nnabla.monitor import Monitor, MonitorSeries, MonitorTimeElapsed

2018-08-16 09:18:59,931 [nnabla][INFO]: Initializing CPU extension...


In [5]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
env = gym.make("Hopper-v2")
state_num = env.observation_space.shape[0]
action_num = env.action_space.shape[0]
hidden_size = 128
max_frames = 200000
num_steps = 20000
batch_size = 32
gamma = 0.99
learning_rate = 1e-3

plot_interval = 1000

Choosing the latest nvidia driver: /usr/lib/nvidia-396, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-396']
Choosing the latest nvidia driver: /usr/lib/nvidia-396, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-396']


In [7]:
#Actor:状態に対して行動の選択確率(方策)を返す
def Actor(state):
    with nn.parameter_scope("Actor"):
        with nn.parameter_scope("Actor-affine1"):
            h = F.relu(PF.affine(state, hidden_size))
        with nn.parameter_scope("Actor-affine2"):
            A = PF.affine(h, action_num)
    return A

#Critic:状態に対して価値を返す
def Critic(state):
    with nn.parameter_scope("Critic"):
        with nn.parameter_scope("Critic-affine1"):
            h = F.relu(PF.affine(state, hidden_size))
        with nn.parameter_scope("Critic-affine2"):
            C = PF.affine(h, 1)
    return C

def get_policy(state):
    State = nn.Variable([1, state_num])
    State.d = state
    Policy = Actor(state)
    Policy.forward()
    return Policy #.d


def get_value(state):
    State = nn.Variable([1, state_num])
    State.d = state
    Value = Critic(state)
    Value.forward()
    return Value.d #.d



In [8]:
#変数定義
State = nn.Variable([batch_size, state_num]) 
Action = nn.Variable([batch_size, 1])
#TODO 損失関数に必要な関数を定義する

#損失関数定義
#TODO Actor-Criticを解くための損失関数の定義を行う
#loss = 

#solver定義
solver = S.Adam(learning_rate)
with nn.parameter_scope("Actor"):
    solver.set_parameters(nn.get_parameters())
with nn.parameter_scope("Critic"):
    solver.set_parameters(nn.get_parameters())
solver.zero_grad()

losses = []
overall_rewards = []
episode_reward = 0

In [9]:
def plot(frame, rewards, losses):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame, np.mean(rewards[-10:])))
    plt.plot(rewards)
    plt.subplot(132)
    plt.title('loss')
    plt.plot(losses)
    plt.show()

In [10]:
state = env.reset()
#nn.clear_parameters()
for frame in range(1, max_frames):
    episode_reward = 0
    for i in range (num_steps):
        #TODO 方策から行動決定
        #action = 
        next_state, reward, done, _ = env.step(action)
        state = next_state
        episode_reward += reward   
        if done:
            state = env.reset()
            overall_rewards.append(episode_reward)  
    loss = compute_gradient()
    losses.append(np.average(loss.d[0]))
    if frame % plot_interval == 0:
        plot(frame, overall_rewards, losses)

NameError: name 'action' is not defined